# 🌳 Decision Predicate Graph (DPG)
This notebook shows a minimal working example of using DPG on a classification dataset.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from dpg.core import DecisionPredicateGraph
from dpg.visualizer import plot_dpg

## 📥 Load your dataset

In [ ]:
# Assumes your dataset is in datasets/custom.csv
# Last column is the target
df = pd.read_csv("../datasets/custom.csv", index_col=0)
X = df.iloc[:, :-1].replace([np.inf, -np.inf], np.nan).fillna(df.mean())
y = df.iloc[:, -1]

## 🧠 Train a Random Forest

In [ ]:
model = RandomForestClassifier(n_estimators=10, random_state=27)
model.fit(X, y)

## 📊 Generate and visualize DPG

In [ ]:
feature_names = X.columns.tolist()
class_names = np.unique(y).astype(str).tolist()

dpg = DecisionPredicateGraph(
    model=model,
    feature_names=feature_names,
    target_names=class_names,
    perc_var=0.0001,
    decimal_threshold=2,
    n_jobs=1
)
dot = dpg.fit(X.values)
dpg_model, nodes_list = dpg.to_networkx(dot)

dpg_metrics = dpg.extract_graph_metrics(dpg_model, nodes_list)
df_nodes = dpg.extract_node_metrics(dpg_model, nodes_list)

plot_dpg("dpg_output.png", dot, df_nodes, dpg_metrics, save_dir="../datasets", communities=True, class_flag=True)